In [1]:
import os

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
import sys


def in_kaggle():
    return "kaggle_web_client" in sys.modules


log.info(f"in_kaggle: {in_kaggle()}")

2023-03-01 07:49:40,041 [INFO] [365646948] in_kaggle: False


In [4]:
##### INSERT SOURCE CODE HERE FOR SUBMISSION #####

# PLACE HOLDER

In [5]:
if in_kaggle():
    !cp -r /kaggle/input/graphnet-and-dependencies/software .
    !pip install -q /kaggle/working/software/dependencies/torch-1.11.0+cu115-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_geometric-2.0.4.tar.gz
    !cd software/graphnet; pip install -q --no-index --find-links="/kaggle/working/software/dependencies" -e .[torch]

    !cp -r /kaggle/input/hydra-core /kaggle/working
    !mv /kaggle/working/hydra-core/antlr4-python3-runtime-4.9.3.tar.gz.tmp /kaggle/working/hydra-core/antlr4-python3-runtime-4.9.3.tar.gz
    !pip install -q /kaggle/working/hydra-core/*

else:
    %cd ..

/home/jovyan/ice-cube


In [6]:
# from omegaconf import OmegaConf

# omega_conf_dir = "config"

# c_main = OmegaConf.load(os.path.join(omega_conf_dir, "main.yaml"))
# c_data = OmegaConf.load(os.path.join(omega_conf_dir, "data.yaml"))
# c = OmegaConf.merge(c_main, c_data)

# c.settings.debug = False
# c.wandb.enabled = False

In [7]:
!python 00-create-db.py settings.in_kaggle={in_kaggle()}

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
graphnet: INFO     2023-03-01 07:49:42 - get_logger - Writing log to logs/graphnet_20230301-074942.log
graphnet: WARNING  2023-03-01 07:49:42 - warn_once - `icecube` not available. Some functionality may be missing.
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")
2023-03-01 07:49:43,943 [INFO][utils] Output directory: ../output/2023-03-01_07-49-43
2023-03-01 07:49:43,943 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-03-01 07:49:43,943 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-03-01 07:49:43,943 [INFO][utils] Fix seed: 440
2023-

In [10]:
!python 09-inference.py settings.in_kaggle={in_kaggle()}

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
graphnet: INFO     2023-03-01 12:45:05 - get_logger - Writing log to logs/graphnet_20230301-124505.log
graphnet: WARNING  2023-03-01 12:45:05 - warn_once - `icecube` not available. Some functionality may be missing.
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")
2023-03-01 12:45:07,785 [INFO][utils] Output directory: ../output/2023-03-01_12-45-07
2023-03-01 12:45:07,785 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-03-01 12:45:07,785 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-03-01 12:45:07,785 [INFO][utils] Fix seed: 440
2023-